In [1]:
import pandas as pd
import numpy as np
#import scipy as sp
import datetime
import sklearn
from sklearn.model_selection import train_test_split

#%matplotlib inline

In [2]:
def oneHotEncoding(df, columna):
  
    one_hot = pd.get_dummies(df[columna])
    df.drop(columna, axis=1, inplace=True)
    df = pd.concat([df, one_hot], axis=1)
    return df

# Cargo CSV

In [3]:
test_final = pd.read_csv('../data/test_final_100k.csv')

In [ ]:
postulantes_educacion = pd.read_csv('../data/datos_navent_fiuba/fiuba_1_postulantes_educacion.csv')
postulantes_educacion = postulantes_educacion.append(pd.read_csv('../data/fiuba_hasta_15_abril/fiuba_1_postulantes_educacion.csv'))
postulantes_educacion = postulantes_educacion.append(pd.read_csv('../data/fiuba_desde_15_abril/fiuba_1_postulantes_educacion.csv'))
postulantes_educacion = postulantes_educacion.reset_index().drop(columns='index')

In [20]:
postulantes_genero_y_edad = pd.read_csv('../data/datos_navent_fiuba/fiuba_2_postulantes_genero_y_edad.csv')
postulantes_genero_y_edad = postulantes_genero_y_edad.append(pd.read_csv('../data/fiuba_hasta_15_abril/fiuba_2_postulantes_genero_y_edad.csv'))
postulantes_genero_y_edad = postulantes_genero_y_edad.append(pd.read_csv('../data/fiuba_desde_15_abril/fiuba_2_postulantes_genero_y_edad.csv'))
postulantes_genero_y_edad = postulantes_genero_y_edad.reset_index().drop(columns='index')

In [ ]:
vistas = pd.read_csv('../data/datos_navent_fiuba/fiuba_3_vistas.csv')
vistas = vistas.append(pd.read_csv('../data/fiuba_hasta_15_abril/fiuba_3_vistas.csv'))
vistas = vistas.append(pd.read_csv('../data/fiuba_desde_15_abril/fiuba_3_vistas.csv'))
vistas = vistas.reset_index().drop(columns='index')

In [4]:
postulaciones = pd.read_csv('../data/datos_navent_fiuba/fiuba_4_postulaciones.csv')
postulaciones = postulaciones.append(pd.read_csv('../data/fiuba_hasta_15_abril/fiuba_4_postulaciones.csv'))
# no tiene postulaciones -> pd.read_csv('data/fiuba_desde_15_abril/')
postulaciones = postulaciones.reset_index().drop(columns='index')

In [ ]:
avisos_online = pd.read_csv('../data/datos_navent_fiuba/fiuba_5_avisos_online.csv')
avisos_online = avisos_online.append(pd.read_csv('../data/fiuba_hasta_15_abril/fiuba_5_avisos_online.csv'))
# no tiene avisos online pd.read_csv('data/fiuba_desde_15_abril/')
avisos_online = avisos_online.reset_index().drop(columns='index')

In [5]:
avisos_detalle = pd.read_csv('../data/datos_navent_fiuba/fiuba_6_avisos_detalle.csv')
avisos_detalle = avisos_detalle.append(pd.read_csv('../data/fiuba_hasta_15_abril/fiuba_6_avisos_detalle.csv'))
avisos_detalle = avisos_detalle.append(pd.read_csv('../data/fiuba_desde_15_abril/fiuba_6_avisos_detalle.csv'))
avisos_detalle = avisos_detalle.append(pd.read_csv('../data/fiuba_6_avisos_detalle_missing_nivel_laboral.csv'))
avisos_detalle = avisos_detalle.reset_index().drop(columns='index')

# Preproceso datos

### Limpio el DF *postulantes_educacion*

In [ ]:
postulantes_educacion.head()

In [ ]:
postulantes_educacion.shape

In [ ]:
postulantes_educacion.isnull().sum()

In [ ]:
postulantes_educacion.isna().sum()

In [ ]:
postulantes_educacion[(postulantes_educacion.duplicated()) & (postulantes_educacion['idpostulante'] == 'NdJl')]

In [ ]:
postulantes_educacion = postulantes_educacion.drop_duplicates(keep='last')

In [ ]:
postulantes_educacion = postulantes_educacion.reset_index().drop(columns='index')

### Limpio el DF *postulantes_genero_y_edad*

In [6]:
postulantes_genero_y_edad.head()

NameError: name 'postulantes_genero_y_edad' is not defined

In [ ]:
postulantes_genero_y_edad.shape

In [ ]:
postulantes_genero_y_edad.isna().sum()

In [ ]:
postulantes_genero_y_edad.isnull().sum()

Luego veremos que hacer con los ids sin fecha de nacimiento, por el momento los dejamos.

In [ ]:
postulantes_genero_y_edad = postulantes_genero_y_edad.drop_duplicates()

In [ ]:
postulantes_genero_y_edad[postulantes_genero_y_edad['idpostulante'] == '0zP6MJa']

In [ ]:
postulantes_genero_y_edad[postulantes_genero_y_edad['idpostulante'] == '0zPvY49']

In [ ]:
#Dejo la ultima aparicion del duplicado, supongo que es el update más reciente
postulantes_genero_y_edad = postulantes_genero_y_edad.drop_duplicates(subset=['idpostulante'],keep='last')

In [ ]:
postulantes_genero_y_edad[postulantes_genero_y_edad['idpostulante'] == '0zP6MJa']

In [ ]:
postulantes_genero_y_edad[postulantes_genero_y_edad['idpostulante'] == '0zPvY49']

In [ ]:
postulantes_genero_y_edad['sexo'].value_counts()

Al igual que con la fecha de nacimiento, por el momento, dejamos los ids con sexo 0.0

In [ ]:
postulantes_genero_y_edad = postulantes_genero_y_edad.reset_index().drop(columns='index')

### Limpio el DF *vistas*

In [ ]:
vistas.head()

In [ ]:
vistas.shape

In [ ]:
vistas.isna().sum()

In [ ]:
vistas.isnull().sum()

In [ ]:
vistas = vistas.drop_duplicates(keep='last')

In [ ]:
vistas = vistas.reset_index().drop(columns='index')

### Limpio el DF *postulaciones*

In [7]:
postulaciones.head()

,idaviso,idpostulante,fechapostulacion
0,1112257047,NM5M,2018-01-15 16:22:34
1,1111920714,NM5M,2018-02-06 09:04:50
2,1112346945,NM5M,2018-02-22 09:04:47
3,1112345547,NM5M,2018-02-22 09:04:59
4,1112237522,5awk,2018-01-25 18:55:03


In [8]:
postulaciones.shape

(8311264, 3)

In [9]:
postulaciones.isna().sum()

idaviso             0
idpostulante        0
fechapostulacion    0
dtype: int64

In [10]:
postulaciones.isnull().sum()

idaviso             0
idpostulante        0
fechapostulacion    0
dtype: int64

In [11]:
postulaciones = postulaciones.drop_duplicates(keep='last')

In [12]:
postulaciones = postulaciones.reset_index().drop(columns='index')

### Limpio el DF *avisos_online*

In [ ]:
avisos_online.head()

In [ ]:
avisos_online.shape

In [ ]:
avisos_online.isna().sum()

In [ ]:
avisos_online.isnull().sum()

In [ ]:
avisos_online = avisos_online.drop_duplicates(keep='last')

In [ ]:
avisos_online = avisos_online.reset_index().drop(columns='index')

### Limpio el DF *avisos_detalle*

In [13]:
avisos_detalle.head()

,idaviso,idpais,titulo,descripcion,nombre_zona,ciudad,mapacalle,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,8725750,1,VENDEDOR/A PROVINCIA DE SANTA FE,<p><strong><strong>Empresa:</strong></strong> ...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Comercial,VENTOR
1,17903700,1,Enfermeras,<p>Solicitamos para importante cadena de farma...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Salud,Farmacias Central Oeste
2,1000150677,1,Chofer de taxi,<p>TE GUSTA MANEJAR? QUERES GANAR PLATA HACIEN...,Capital Federal,NaN,Empedrado 2336,Full-time,Senior / Semi-Senior,Transporte,FAMITAX SRL
3,1000610287,1,CHOFER DE CAMIONETA BAHIA BLANCA - PUNTA ALTA,<p><strong>Somos una empresa multinacional que...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Transporte,Wurth Argentina S.A
4,1000872556,1,Operarios de Planta - Rubro Electrodomésticos,<p><strong>OPERARIOS DE PLANTA</strong></p><p>...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Producción,ELECTRO OUTLET SRL


In [14]:
avisos_detalle.shape

(45969, 11)

In [15]:
avisos_detalle['idaviso'].duplicated().sum()

20681

In [16]:
avisos_detalle = avisos_detalle.drop_duplicates(subset=['idaviso'],keep='last')

In [17]:
avisos_detalle = avisos_detalle.reset_index().drop(columns='index')

# Preprocesamiento

In [21]:
# Agrego la edad de los postulantes
postulantes_genero_y_edad['fechanacimiento'] = pd.to_datetime(postulantes_genero_y_edad['fechanacimiento'],errors='coerce')
postulantes_genero_y_edad['edad'] = datetime.datetime.now().year - postulantes_genero_y_edad['fechanacimiento'].dt.year

In [22]:
postulantes_genero_y_edad.head()

,idpostulante,fechanacimiento,sexo,edad
0,NM5M,1970-12-03,FEM,48.0
1,5awk,1962-12-04,FEM,56.0
2,ZaO5,1978-08-10,FEM,40.0
3,NdJl,1969-05-09,MASC,49.0
4,eo2p,1981-02-16,MASC,37.0


In [23]:
# Despues veo que hago con los que no tienen edad
postulantes_genero_y_edad.isna().sum()

idpostulante           0
fechanacimiento    34320
sexo                   0
edad               34320
dtype: int64

In [24]:
# Separo gente postulada de no postulada
# Tupla (idpostulante,idaviso)
#tupla_postulada = postulaciones.drop(columns='fechapostulacion')

In [25]:
# Separa la gente postulada, luego los busco en el DF que junta todos los postulantes
# y me quedo con los opuestos, los no postulados.
# esto me va a servir para samplear
# GENTE NO POSTULADA
gente_no_postulada = postulantes_genero_y_edad[-postulantes_genero_y_edad['idpostulante'].\
                                               isin(postulaciones['idpostulante'].value_counts().index)]
gente_no_postulada = gente_no_postulada.reset_index(drop=True)

In [26]:
gente_no_postulada['sexo'].value_counts()

FEM           79306
MASC          75145
NO_DECLARA    18064
0.0               6
Name: sexo, dtype: int64

In [27]:
# Pocos datos con sexo invalido, los desecho ya que son los no postulados
gente_no_postulada = gente_no_postulada.drop(gente_no_postulada[gente_no_postulada['sexo'] == '0.0'].index)

In [34]:
# GENTE POSTULADA
gente_postulada = postulantes_genero_y_edad[postulantes_genero_y_edad['idpostulante'].\
                                               isin(postulaciones['idpostulante'].value_counts().index)]
gente_postulada = gente_postulada.reset_index(drop=True)

In [35]:
gente_postulada['sexo'].value_counts()

FEM           312173
MASC          280093
NO_DECLARA     15232
0.0                1
Name: sexo, dtype: int64

In [36]:
# Reemplazo el sexo invalido por NO_DECLARA
gente_postulada.loc[gente_postulada['sexo'] == '0.0','sexo'] = 'NO_DECLARA'

In [31]:
# Dropeo los datos invalidos de los no postulados (se podria buscar alguna forma de salvarlos)
gente_no_postulada = gente_no_postulada.dropna()
gente_no_postulada = gente_no_postulada.reset_index(drop=True)
gente_no_postulada['edad'] = gente_no_postulada['edad'].astype('int32')

In [38]:
# En un principio los relleno con la media de los postulantes
gente_postulada.loc[gente_postulada['edad'].isna(),'edad'] = int(postulantes_genero_y_edad['edad'].mean())
gente_postulada['edad'] = gente_postulada['edad'].astype('int32')

In [39]:
gente_postulada.isna().sum()

idpostulante           0
fechanacimiento    15885
sexo                   0
edad                   0
dtype: int64

# Funciones de utilidad

In [79]:
# fijarse de balancear el sample con 50% de postulados y no postulados

# Generador de sample
# el id tiene que estar en la columna 0
# Recibe el DF de idpostulante e idaviso
def generador_de_sample_df(idaviso, idpostulante, n_sample, ciclos):
    idpostulante = idpostulante.drop(columns=idpostulante.columns[1:])
    idaviso = idaviso.drop(columns=idaviso.columns[1:])
    idpostulante['tem'] = 1
    idaviso['tem'] = 1
    sample_append = pd.DataFrame(columns=[idpostulante.columns[0],'tem',idaviso.columns[0]])
    for i in range(0,ciclos):
        sample_idpostulante = idpostulante.sample(n=n_sample,random_state=i)
        sample_idaviso = idaviso.sample(n=n_sample,random_state=i)
        sample_merge = sample_idaviso.merge(sample_idpostulante)
        sample_append = sample_append.append(sample_merge)
    sample_append = sample_append.drop(columns='tem')
    sample_append = sample_append.reset_index(drop=True)
    return sample_append

In [76]:
# Recibe la series de idpostulante e idaviso
def generador_de_sample_serie(idaviso, idpostulante, n_sample, ciclos):
    idpostulante = pd.DataFrame(idpostulante)
    idaviso = pd.DataFrame(idaviso)
    idpostulante['tem'] = 1
    idaviso['tem'] = 1
    sample_append = pd.DataFrame(columns=[idpostulante.columns[0],'tem',idaviso.columns[0]])
    for i in range(0,ciclos):
        sample_idpostulante = idpostulante.sample(n=n_sample,random_state=i)
        sample_idaviso = idaviso.sample(n=n_sample,random_state=i)
        sample_merge = sample_idaviso.merge(sample_idpostulante)
        sample_append = sample_append.append(sample_merge)
    sample_append = sample_append.drop(columns='tem')
    sample_append = sample_append.reset_index(drop=True)
    return sample_append

# Funciones feature

In [ ]:
# REVISAR
# duelve un DF booleano
def feature_se_postulo_alguna_vez(df_1,df_2):
    idpostulante = df_2['idpostulante'].value_counts().index
    idaviso = df_2['idaviso'].value_counts().index
    feature = df_1['idpostulante'].isin(idpostulante) & df_1['idaviso'].isin(idaviso)
    #feature = feature.iloc[:,0] & feature.iloc[:,1]
    return feature

In [ ]:
def feature_postulante_se_postulo_al_aviso(idpostulante,idaviso,postulaciones):
    